In [72]:
import networkx as nx

def dfs(G, s):
    n = len(G)
    colors = ['black']*n
    frame_colors = [colors.copy()]
    widths = [1]*len(G.edges)
    frame_widths = [widths.copy()]
    for i,e in enumerate(G.edges):
        G.edges[e]['index'] = i

    def add_frame():
        frame_colors.append(colors.copy())
        frame_widths.append(widths.copy())

    def aux(u):
        colors[u] = 'red'
        add_frame()
        colors[u] = 'orange'
        for v in G.neighbors(u):
            if colors[v] == 'black':
                widths[G[u][v]['index']] = 5
                aux(v)
        colors[u] = 'green'
        add_frame()

    colors[s] = 'yellow'
    aux(s)
    return frame_colors, frame_widths

In [73]:
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML

def animate_graph(G, colors, widths, **kwargs):
    fig, ax = plt.subplots(figsize=(10,10))
    pos = nx.spring_layout(G)
    plt.close()
    
    def update(frame):
        ax.clear()
        p = len(colors[frame]) #width=[100]*p, 
        nx.draw(G, ax=ax, pos=pos, node_size=200, width = widths[frame], node_color = colors[frame], **kwargs)
    
    ani = matplotlib.animation.FuncAnimation(fig, update, frames=len(colors), interval=1000, repeat=True)
    return ani.to_jshtml()

In [74]:
colors, widths = dfs(G, 0)
s = animate_graph(G, colors, widths)
HTML(s)